In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import sum,avg,max,count

In [0]:
spark = SparkSession.builder.master("local[*]").appName("Datamanipulation").getOrCreate()

In [0]:
df = spark.read.table("sample___eu_superstore_1_csv")

In [0]:
data = df.filter(df["Country"] == "France")
data.count()

Out[19]: 2827

In [0]:
data.filter(df["Profit"] > 0).count()

Out[20]: 2329

In [0]:
dis_set = df.select(["Discount"]).distinct()
dis_set.count()
dis_set.show()

+--------+
|Discount|
+--------+
|    null|
|       0|
+--------+



In [0]:
df.groupBy("Discount").agg({'Profit': 'sum'}).orderBy('sum(Profit)').show()

+--------+------------------+
|Discount|       sum(Profit)|
+--------+------------------+
|    null|-10976.78881773213|
|       0| 383806.5302368067|
+--------+------------------+



In [0]:
cust_profit = df.groupBy(["Customer ID","Customer Name"]).agg({'Profit': 'sum'}).orderBy('Sum(Profit)',ascending = False)
cust_profit.show(5)

+-----------+-----------------+------------------+
|Customer ID|    Customer Name|       sum(Profit)|
+-----------+-----------------+------------------+
|   SP-20920|     Susan Pistek| 4974.513064682484|
|   PJ-18835|    Patrick Jones|3986.0040826797485|
|   PO-18865|Patrick O'Donnell| 3778.197093129158|
|   EB-13840|    Ellis Ballard| 3459.660038240254|
|   MG-18145|  Mike Gockenbach|3144.4440593719482|
+-----------+-----------------+------------------+
only showing top 5 rows



In [0]:
cust_profit.collect()

Out[24]: [Row(Customer ID='SP-20920', Customer Name='Susan Pistek', sum(Profit)=4974.513064682484),
 Row(Customer ID='PJ-18835', Customer Name='Patrick Jones', sum(Profit)=3986.0040826797485),
 Row(Customer ID='PO-18865', Customer Name="Patrick O'Donnell", sum(Profit)=3778.197093129158),
 Row(Customer ID='EB-13840', Customer Name='Ellis Ballard', sum(Profit)=3459.660038240254),
 Row(Customer ID='MG-18145', Customer Name='Mike Gockenbach', sum(Profit)=3144.4440593719482),
 Row(Customer ID='ER-13855', Customer Name='Elpida Rittenbach', sum(Profit)=2899.805934727192),
 Row(Customer ID='LB-16795', Customer Name='Laurel Beltran', sum(Profit)=2892.717029929161),
 Row(Customer ID='RB-19330', Customer Name='Randy Bradley', sum(Profit)=2864.600984307006),
 Row(Customer ID='JH-15820', Customer Name='John Huston', sum(Profit)=2860.1025133132935),
 Row(Customer ID='MS-17770', Customer Name='Maxwell Schwartz', sum(Profit)=2795.8799901008606),
 Row(Customer ID='BV-11245', Customer Name='Benjamin Ven

In [0]:
cust_profit.count()

Out[25]: 795

In [0]:
df = df.withColumn("Orginal Value of Sale",df['Sales']/(1-df['Discount']))
df.show()

+------+---------------+----------+---------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+---------------------+
|Row ID|       Order ID|Order Date|Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|Orginal Value of Sale|
+------+---------------+----------+---------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+---------------------+
|     1|ES-2017-1311038|      null|     null|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|

In [0]:
df = df.withColumn("Amount of discount",df['Orginal Value of Sale'] - df['Sales'])
df.select("Sales","Amount of discount").show()

+--------+------------------+
|   Sales|Amount of discount|
+--------+------------------+
|    79.2|               0.0|
|  388.92|               0.0|
|   35.19|               0.0|
|   50.94|               0.0|
|  307.44|               0.0|
|   122.4|               0.0|
|  413.82|               0.0|
|  428.22|               0.0|
| 3979.29|               0.0|
|   43.56|               0.0|
|   25.26|               0.0|
|2443.905|              null|
|   12.21|               0.0|
|2167.296|              null|
| 138.105|              null|
| 128.385|              null|
|  690.12|               0.0|
|    8.16|               0.0|
|  347.88|               0.0|
| 575.505|              null|
+--------+------------------+
only showing top 20 rows



In [0]:
count_of_gain = df.groupBy("Discount").agg({'Amount of discount': 'sum'})
count_of_gain.show()

+--------+-----------------------+
|Discount|sum(Amount of discount)|
+--------+-----------------------+
|    null|                   null|
|       0|                    0.0|
+--------+-----------------------+



In [0]:
count_of_gain = df.groupBy("Discount").agg({'Amount of discount': 'sum'}).orderBy("sum(Amount of discount)",ascending =False)
count_of_gain.show()

+--------+-----------------------+
|Discount|sum(Amount of discount)|
+--------+-----------------------+
|       0|                    0.0|
|    null|                   null|
+--------+-----------------------+



In [0]:
df.select(sum("Profit")+sum("Amount of discount")).show()

+---------------------------------------+
|(sum(Profit) + sum(Amount of discount))|
+---------------------------------------+
|                      372829.7414190746|
+---------------------------------------+



In [0]:
df.select((sum("Profit")+sum("Amount of discount"))-sum("Profit")).show()

+-------------------------------------------------------+
|((sum(Profit) + sum(Amount of discount)) - sum(Profit))|
+-------------------------------------------------------+
|                                                    0.0|
+-------------------------------------------------------+



In [0]:
df.createOrReplaceTempView("superstore")

In [0]:
spark.sql("SELECT Country , SUM(Profit)/SUM(Sales) AS profit_ratio FROM superstore GROUP BY Country").show()

+--------------+-------------------+
|       Country|       profit_ratio|
+--------------+-------------------+
|        Sweden|-0.5745674277080725|
|       Germany|0.17066792085089796|
|        France|0.12693568250632956|
|       Belgium|0.23508766322655358|
|       Finland| 0.1886429656028743|
|         Italy|0.06844355261600243|
|        Norway| 0.2517747558357422|
|         Spain|0.18941580526198554|
|       Denmark| -0.495719004670541|
|       Ireland|-0.4442667746327458|
|   Switzerland|0.29092012336571044|
|      Portugal| -0.576166236329489|
|       Austria|0.26419087935813323|
|United Kingdom|0.21170103628484374|
|   Netherlands|-0.5298342848078443|
+--------------+-------------------+



In [0]:
spark.sql("SELECT Country, SUM(Profit) , SUM(Profit)/SUM(Sales) AS profit_ratio FROM superstore GROUP BY Country ORDER BY SUM(Profit) DESC ").show()

+--------------+-------------------+-------------------+
|       Country|        sum(Profit)|       profit_ratio|
+--------------+-------------------+-------------------+
|United Kingdom| 111900.15013868175|0.21170103628484374|
|        France| 109029.00298916548|0.12693568250632956|
|       Germany| 107322.82054615906|0.17066792085089796|
|         Spain|  54390.11980030127|0.18941580526198554|
|       Austria| 21442.259948462248|0.26419087935813323|
|         Italy| 19828.758236613125|0.06844355261600243|
|       Belgium| 11572.589964687824|0.23508766322655358|
|   Switzerland| 7237.4700926840305|0.29092012336571044|
|        Norway|  5167.770015597343| 0.2517747558357422|
|       Finland|  3905.729992687702| 0.1886429656028743|
|       Denmark| -4282.047019600868| -0.495719004670541|
|       Ireland| -7392.381024058908|-0.4442667746327458|
|      Portugal| -8703.060049800202| -0.576166236329489|
|        Sweden|-17519.367032960057|-0.5745674277080725|
|   Netherlands| -41070.0751795